In [1]:
import datetime

In [2]:
class Vert:
    def __init__(self):
        self.edges = dict()
        self.attrs = dict()
        
    def __repr__(self):
        return 'edges:' + str(self.edges) + ', attrs:' + str(self.attrs)

In [3]:
minTime, maxTime = (1235000177, 1244362062)
timeRange = maxTime - minTime

In [98]:
import csv

G = dict()

with open('stackoverflow.csv') as so_file:
    reader = csv.reader(so_file)
    
    next(reader)
    for row in reader:
    
        # Asker ID
        i = row[2]
        # Answerer ID
        j = row[3]
        # Answer time
        at = int(row[11])
        # Tags as a list
        tags = row[5].split(',')
        
        if at < minTime:
            minTime = at
            
        if maxTime < at:
            maxTime = at
        
        ignoredUsers = ['NA', '0', '1']
        if i in ignoredUsers or j in ignoredUsers:
            continue
        
        if not i in G:
            G[i] = Vert()
    
        if not j in G:
            G[j] = Vert()
        
        if not j in G[i].edges or at > G[i].edges[j]:
            G[i].edges[j] = (at - minTime) / timeRange
        
        if not i in G[j].edges or at > G[j].edges[i]:
            G[j].edges[i] = (at - minTime) / timeRange
        
        for tag in tags:
            if not tag in G[i].attrs:
                G[i].attrs[tag] = 0
        
            if not tag in G[j].attrs:
                G[j].attrs[tag] = 0
            
            G[i].attrs[tag] += 1
            G[j].attrs[tag] += 1
    
print()
    
#usersRemoved = list(filter(lambda p: p[1] <= 3, sorted([(x[0], len(x[1].edges)) for x in G.items()],key=lambda y: y[1], reverse=True)))[2:]
usersRemoved = list(filter(lambda p: len(G[p].edges) < 3 or len(G[p].edges) > 300, G.keys()))

print(len(usersRemoved))
print(len(G))

inverseRemoved = list()
for rm in usersRemoved:
    for irm in G[rm].edges.keys():
        inverseRemoved.append( (irm, rm) )
    G.pop(rm)
    
for irm, rm in inverseRemoved:
    if irm in G:
        if rm in G[irm].edges:
            G[irm].edges.pop(rm)
            if len(G[irm].edges) <= 0:
                G.pop(irm)
    
print(len(G))
    
print()    

ranges = {
    (0, .1): 0,
    (.1, .2): 0,
    (.2, .3): 0,
    (.3, .4): 0,
    (.4, .5): 0,
    (.4, .5): 0,
    (.5, .6): 0,
    (.6, .7): 0,
    (.7, .8): 0,
    (.8, .9): 0,
    (.9, 1.1): 0
}
    
for k,v in G.items():
    for e in v.edges.values():
        for rs, re in ranges.keys():
            if rs <= e and e < re:
                ranges[(rs, re)] += 1
            


10357
11837
531



In [99]:
ranges

{(0, 0.1): 110,
 (0.1, 0.2): 124,
 (0.2, 0.3): 130,
 (0.3, 0.4): 144,
 (0.4, 0.5): 124,
 (0.5, 0.6): 152,
 (0.6, 0.7): 150,
 (0.7, 0.8): 142,
 (0.8, 0.9): 134,
 (0.9, 1.1): 158}

In [100]:
[]

[]

In [101]:
minTime, maxTime

(1235000177, 1244362062)

In [102]:
len(G.items()) # excluding '0' and '1' goes from 26883 to 11837

531

In [103]:
def commonNeighbors(cl, v):
    n_v = set(G[v].edges.keys())

    return len(cl.intersection(n_v)) / len(cl.union(n_v))

In [104]:
def getNextNode(cluster, clNeighbors):
    edgeWeightSums = [(e[0], sum(G[e[0]].edges[o] for o in e[1])) for e in [(n, set(cluster).intersection(set(G[n].edges.keys()))) for n in clNeighbors ]]
    sortedEdgeWeightSums = sorted(edgeWeightSums, key=lambda e: e[1], reverse=True)
    topEdgesByWeight = list(filter(lambda x: x[1] == sortedEdgeWeightSums[0][1], sortedEdgeWeightSums))
    return sorted([(t[0], t[1], len(set(G[t[0]].edges.keys()).difference(cluster).intersection(clNeighbors))) for t in topEdgesByWeight], key=lambda e: e[2], reverse=True)[0]

In [105]:
log = list()

def densityCluster(t_w):
    # 5 - 7
    nodeExists = dict()
    for node, edges_node in G.items():
        nodeExists[node] = True
    
    # 8
    clusters = list()
    
    # 9
    nodeDegree = dict()
    # 10 - 12
    for node, edges_node in G.items():
        nodeDegree[node] = len(edges_node.edges)
        
    
    for nId, __n_edges in sorted([(x[0], len(x[1].edges)) for x in G.items()],key=lambda y: y[1], reverse=True):
        node = str(nId)
        if nodeExists[node]:
            c = set()
            maxDegree = node
            nodeDegree[maxDegree] = -1
            nodeExists[maxDegree] = False
            neighbors = set(G[maxDegree].edges.keys())
            c.add(maxDegree)
            
            #for neighbor, edge_weight in neighbors:
                #print(float(edge_weight) + commonNeighbors(node, neighbor))
            
            #print(neighbors)
            
            while len(neighbors) > 0:
                neighbor, edge_weight, no_neighbors = getNextNode(c, neighbors)
                neighbors.remove(neighbor)
                
                if not nodeExists[neighbor]:
                    continue
                
                l = (edge_weight, no_neighbors)
                log.append(l)
                #print(l)
                
                if (edge_weight + commonNeighbors(c, neighbor)) >= t_w:
                    c.add(neighbor)
                    nodeExists[neighbor] = False
                    nodeDegree[neighbor] = -1
                    
                    newNeighbors = list(filter(lambda x: nodeExists[x], G[neighbor].edges.keys()))
                    neighbors.update( newNeighbors )
                                    
            #print(neighbors)
            
            clusters.append(c)
            
    return clusters


In [115]:
print(datetime.datetime.now())
cs = sorted(densityCluster(1), key=lambda x: len(x), reverse=True)
cs = list(filter(lambda q: len(q) > 1, cs))
print(datetime.datetime.now())
[len(x) for x in cs]

2019-11-26 01:13:37.522256
2019-11-26 01:13:37.647781


[42, 16, 8, 6, 5, 5, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2]

In [119]:
for c in cs:
    topics = dict()
    
    for o in c:
        for t in G[o].attrs.keys():
            if not t in topics:
                topics[t] = 0
            topics[t] += 1
    
    print(sorted(topics.items(), key=lambda t: t[1], reverse=True)[:5])
    

[('best-practices', 29), ('c#', 28), ('ûnet', 27), ('subjective', 27), ('c++', 20)]
[('best-practices', 8), ('subjective', 6), ('java', 6), ('language-agnostic', 6), ('c++', 6)]
[('subjective', 5), ('c#', 4), ('c++', 4), ('sqlserver', 3), ('beginner', 3)]
[('subjective', 4), ('programming-languages', 4), ('ûnet', 3), ('python', 3), ('c#', 3)]
[('aspûnet', 3), ('beginner', 3), ('ûnet', 3), ('best-practices', 2), ('xml', 2)]
[('javascript', 3), ('best-practices', 3), ('c#', 3), ('subjective', 3), ('svn', 3)]
[('c#', 3), ('language-agnostic', 3), ('unit-testing', 2), ('ûnet', 2), ('job-interview', 2)]
[('java', 4), ('best-practices', 3), ('generics', 2), ('void', 2), ('delphi', 2)]
[('best-practices', 3), ('sql', 2), ('sqlserver', 2), ('game-development', 2), ('resources', 2)]
[('subjective', 3), ('c', 2), ('c++', 2), ('multithreading', 2), ('threads', 2)]
[('personal-development', 3), ('subjective', 3), ('java', 3), ('database', 3), ('management', 2)]
[('subjective', 3), ('python', 2), (